## Setup

If you are connecting in a regular jupyter notebook and have activated our `ag-tutorial` environment then you do not need to run the following two cells to install `agraph-python` and `pandas`. However if you are in a **Google Colab** please run the following cell.

In [ ]:
%pip install agraph-python pycurl

We start by setting the environment variables based on your server installation.
If you are connecting on a cloud server, you do not need to change `AGRAPH_PORT` or `AGRAPH_USER`, but you do need to change `AGRAPH_HOST` and `AGRAPH_PASSWORD` to fit your server. We have left some variables in place as an example of what to look for, but they need to be replaced with your values.
If you are connecting to a local installation the host will most likely be `localhost` and whatever port you installed AllegroGraph on (standard is `10035`). Then your `AGRAPH_USER` and `AGRAPH_PASSWORD` need to be added.

In [1]:
import os

os.environ['AGRAPH_HOST'] = 'https://ag197y8xsj2epl2e.allegrograph.cloud' #add your AllegroGraph Cloud url here (starting with https, copy till the end of allegrograph.cloud)
os.environ['AGRAPH_PORT'] = '443' #Agraph is listening at port 443, you do not need to change this
os.environ['AGRAPH_USER'] = 'admin' #your username should be 'admin', you do not need to change this
os.environ['AGRAPH_PASSWORD'] = 'GrMEKDvQFaN2bkrHJeiCbv' #Add your password here as a string

# Adding and Querying Data with Python in AllegroGraph

We start by making a connection object to a repository at the specified server location.

In [2]:
from franz.openrdf.connect import ag_connect

conn = ag_connect('agraph-example', clear=True)

Get the size of the repository

In [3]:
conn.size()

0

### Namespaces
First we define a new namespace that we will use in our local python code.

In [5]:
f = conn.namespace('http://franz.com/')

Once we've defined this we can use this namespace object to create any part of a triple.

In [6]:
f.example

<http://franz.com/example>

### Adding triples
Now we want to add our first triple using this namespace.

In [7]:
conn.add(f.being1, f.friendOf, f.being2)
conn.size()

1

This predicate is an example of an object property, establishing a relationship between two objects (represented as URIs). Now, we want to add some data properties (descriptive data) to the two objects, and show how we can add multiple triples at a time if we add them to a list.

In [8]:
triples = []
triples.append((f.being1, f.name, "Frodo"))
triples.append((f.being2, f.name, "Gandalf"))
conn.addTriples(triples)
conn.size()

3

### Querying the data
Now we query for the existing data in the graph. 

In [9]:
from franz.openrdf.query.query import QueryLanguage

query_string = """SELECT ?s ?p ?o  
                  WHERE {?s ?p ?o . }"""
result = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate()
with result:
    for binding_set in result:
        s = binding_set.getValue('s')
        p = binding_set.getValue('p')
        o = binding_set.getValue('o')
        print(s, p, o)

<http://franz.com/being1> <http://franz.com/friendOf> <http://franz.com/being2>
<http://franz.com/being1> <http://franz.com/name> "Frodo"
<http://franz.com/being2> <http://franz.com/name> "Gandalf"


This is probably the most verbose method of displaying the output of the query, we also frequently directly place the results in a Pandas dataframe.

In [10]:
df = conn.executeTupleQuery(query_string).toPandas()
df

,s,p,o
0,<http://franz.com/being1>,<http://franz.com/friendOf>,<http://franz.com/being2>
1,<http://franz.com/being1>,<http://franz.com/name>,Frodo
2,<http://franz.com/being2>,<http://franz.com/name>,Gandalf


### Adding labels and types
Let's look at the data in Gruff...

![beings-without-types](images/beings-without-types.png)

It is also very simple to add `rdf` types, and `rdfs` labels (among other things), so let's make this picture a bit nicer (and the data more expressive and usable)

In [11]:
from franz.openrdf.vocabulary import RDF, RDFS

triples = []
triples.append((f.being1, RDF.TYPE, f.Hobbit))
triples.append((f.being2, RDF.TYPE, f.Wizard)) #Istari for the real nerds out there
triples.append((f.being1, RDFS.LABEL, "Frodo"))
triples.append((f.being2, RDFS.LABEL, "Gandalf"))
conn.addTriples(triples)

![beings-with-types](images/beings-with-types.png)

### Literals
We can also add different types of literals and then query for those values

In [12]:
from franz.openrdf.vocabulary.xmlschema import XMLSchema
from datetime import datetime, date, time

triples = []
triples.append((f.being1, f.age, conn.createLiteral(50, datatype=XMLSchema.INT)))
triples.append((f.being1, f.weight, conn.createLiteral(100.5, datatype=XMLSchema.FLOAT)))
triples.append((f.being1, f.dob, date(1994, 10, 11)))
conn.addTriples(triples)

First, before we start quering the graph with more specific queries we need to declare the namespace to the server as well

In [13]:
conn.setNamespace('f', 'http://franz.com/')

In [14]:
query_string = """
    select ?being ?age ?weight ?dob where {
        ?being  f:age ?age ;
                f:weight ?weight ;
                f:dob ?dob . } """
result = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate()
with result:
    for binding_set in result:
        being = binding_set.getValue('being')
        age = binding_set.getValue('age')
        weight = binding_set.getValue('weight')
        dob = binding_set.getValue('dob')
        print(being, type(being))
        print(age, type(age))
        print(weight, type(weight))
        print(dob, type(dob))

<http://franz.com/being1> <class 'franz.openrdf.model.value.URI'>
"50"^^<http://www.w3.org/2001/XMLSchema#int> <class 'franz.openrdf.model.literal.Literal'>
"1.005E2"^^<http://www.w3.org/2001/XMLSchema#float> <class 'franz.openrdf.model.literal.Literal'>
"1994-10-11"^^<http://www.w3.org/2001/XMLSchema#date> <class 'franz.openrdf.model.literal.Literal'>


Now we will run this same query but grab the actual desired values.

In [15]:
result = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate()
with result:
    for binding_set in result:
        being = binding_set.getValue('being')
        age = binding_set.getValue('age').intValue()
        weight = binding_set.getValue('weight').floatValue()
        dob = binding_set.getValue('dob').dateValue()
        print(being, being.namespace, being.localname, type(being))
        print(age, type(age))
        print(weight, type(weight))
        print(dob, type(dob))

<http://franz.com/being1> http://franz.com/ being1 <class 'franz.openrdf.model.value.URI'>
50 <class 'int'>
100.5 <class 'float'>
1994-10-11 <class 'datetime.date'>


When you query and place the results in a dataframe it automatically converts literals into the correct python data types, but it doesn't do that for URIs since they are not an accepted pandas datatype

In [16]:
df = conn.executeTupleQuery(query_string).toPandas()
df

,being,age,weight,dob
0,<http://franz.com/being1>,50,100.5,1994-10-11


In [17]:
print(f"Being: {type(df['being'][0])}, age: {type(df['age'][0])}, weight: {type(df['weight'][0])}, dob: {type(df['dob'][0])}")

Being: <class 'str'>, age: <class 'numpy.int64'>, weight: <class 'numpy.float64'>, dob: <class 'datetime.date'>


### Query manipulation with f-strings

In [18]:
def find_values_per_id(identifier):
    query_string = f"select ?predicate ?object where {{ f:being{str(identifier)} ?predicate ?object . }}"
    df = conn.executeTupleQuery(query_string).toPandas()
    return df

find_values_per_id(1)

,predicate,object
0,<http://franz.com/dob>,1994-10-11
1,<http://franz.com/weight>,100.5
2,<http://franz.com/age>,50
3,<http://www.w3.org/2000/01/rdf-schema#label>,Frodo
4,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Hobbit>
5,<http://franz.com/name>,Frodo
6,<http://franz.com/friendOf>,<http://franz.com/being2>


### Adding a graph
It is also very simple to add a fourth element to any triples

In [19]:
conn.add(f.being2, f.age, conn.createLiteral(2000, datatype=XMLSchema.INT), f.Tolkien)

Note the fourth element of the triple at the right most part of the screen

![beings-with-types](images/beings-with-graphs.png)

### Deleting duplicates
First let's see how many triples are currently in the graph

In [20]:
conn.size()

11

Now let's add some new triples and add some duplicates as well. 

In [21]:
triples = []
triples.append((f.being2, f.hasRing, f.ring1))
triples.append((f.ring1, f.ringName, "Narya"))
triples.append((f.ring1, f.ringName, "Narya"))
conn.addTriples(triples)
conn.size()

14

Now we delete the duplicates

In [22]:
conn.deleteDuplicates(mode='spo')
conn.size()

13

### Exporting data and adding external files
First we export the data

In [23]:
conn.getStatements(output='tolkien-statements.nq')

Then we clear the database:

In [24]:
conn = ag_connect('agraph-example', clear=True)
conn.setNamespace('http://franz.com/')
conn.size()

0

Then we add the file back in, and also we add a fourth element to all triples

In [25]:
conn.addFile('tolkien-statements.nq', context=f.Tolkien)
conn.size()

13

Now we query for all data and show the `f:Tolkien` is the fourth element of every triple

In [26]:
query_string = "select ?s ?p ?o ?g where { graph ?g { ?s ?p ?o . } }"
df = conn.executeTupleQuery(query_string).toPandas()
df

,s,p,o,g
0,<http://franz.com/being1>,<http://franz.com/friendOf>,<http://franz.com/being2>,<http://franz.com/Tolkien>
1,<http://franz.com/being1>,<http://franz.com/name>,Frodo,<http://franz.com/Tolkien>
2,<http://franz.com/being2>,<http://franz.com/name>,Gandalf,<http://franz.com/Tolkien>
3,<http://franz.com/being1>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Hobbit>,<http://franz.com/Tolkien>
4,<http://franz.com/being2>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Wizard>,<http://franz.com/Tolkien>
5,<http://franz.com/being1>,<http://www.w3.org/2000/01/rdf-schema#label>,Frodo,<http://franz.com/Tolkien>
6,<http://franz.com/being2>,<http://www.w3.org/2000/01/rdf-schema#label>,Gandalf,<http://franz.com/Tolkien>
7,<http://franz.com/being1>,<http://franz.com/age>,50,<http://franz.com/Tolkien>
8,<http://franz.com/being1>,<http://franz.com/weight>,100.5,<http://franz.com/Tolkien>
9,<http://franz.com/being1>,<http://franz.com/dob>,1994-10-11,<http://franz.com/Tolkien>


It is also possible to export the result of a SPARQL query to various file types, and not worry about having to export the entire database

In [27]:
query_string = "select ?being ?age where { ?being f:age ?age . }"
conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate(output='age-query.csv')

For more info about using python with AllegroGraph please check out the [documentation](https://franz.com/agraph/support/documentation/current/python/api.html)